# FIT5196 Assessment 4
#### Student Name: Aditya Manoj Kunagu Venkata, Vivek Veeri
#### Student ID:  30348013, 30081092

Date: 27/10/2019

Version: 1.0

Environment: Python 3.7.0 (64-bit)

Libraries used:
* pandas
* numpy
* matplotlib.pyplot as plt
* from zipfile import ZipFile
* os
* re
* geopandas as gpd
* from shapely.geometry import Point,shape
* from shapely.geometry.polygon import Polygon
* from math import sin, cos, sqrt, radians, atan2
* from datetime import datetime
* csv
* requests
* from urllib.request import urlopen
* from bs4 import BeautifulSoup
* from sklearn.linear_model import LinearRegression
* from sklearn.model_selection import train_test_split
* from sklearn import preprocessing
* from sklearn.metrics import mean_squared_error, r2_score
* math
* ast


##  Introduction
 This assignment comprises of the integrate several datasets into one single schema and find and fix possible problems in the data.. We are provided with six input files, out of which two are in `.zip`, one `.xlsx`, one `.txt`, one `.xml`& one in __[`.csv`_] (Comma Separated Values) file formats.
 
Main task is to integrate the contents of the data in the following files.

**1. Data Integration**

**2. Data Reshaping**
    
More details for each task will be given in the following sections.

## Methodology

For completing the given tasks, we had used the following steps.

**1. Data Integration**
Here we had integrated the given datasets from the given input files & scraping the data from websites to generate the output schema as follows:

 - **ID** - Gives the unique ID of the property.
 - **Address** - Gives the address of the property.
 - **Suburb** - Gives the suburb name of the property which is calculated from `Vic_suburb_boundary.zip.`, using lat & lon from the given data.
 - **Price** - Gives the price of the property.
 - **Type** - Gives the type of the property.
 - **Date** - Gives the date of the property sold.
 - **Rooms** - Gives the number of bedrooms present in the property.
 - **Bathroom** - Gives the number of bathrooms present in the property.
 - **Car** - Gives the number of parking spaces present in the property.
 - **LandSize** - Gives the area of the property.
 - **Age** - Gives the age of the property during selling time.
 - **Latitude** - Gives the latitude of the property.
 - **Longitude** - Gives the longitude of the property.
 - **train_station_id** - Gives the closest train station from the property which has a direct trip to Southern Cross Station.
 - **distance_to_train_station** - Gives the direct distance from the closest train station to the property.
 - **travel_min_to_CBD** - Gives the average time taken by the direct train which has a direct trip to Southern Cross Station on weekdays taht departs between 7 to 9.30 AM
 - **over_priced?** - Gives the boolean feature whether the price of the property is higher than median price of similar properties (with respect to bedrooms, bathrooms, parking_space, and property_type attributes) in the same suburb on the year of selling.
 - **crime_A_average** - Gives the average of type A crime in the local government area where the property is situated.
 - **crime_B_average** - Gives the average of type B crime in the local government area where the property is situated.
 - **crime_C_average** - Gives the average of type C crime in the local government area where the property is situated.
 - **closest_primary_school** - Gives the name of the closest primary school to the property.
 - **distance_to_closest_primary** - Gives the direct distance between the property and the closest primary school.
 - **primary_school_ranking** - Gives the ranking of the closest primary school to the property which is scraped from the website. If the school is not listed, append the value as "not ranked".
 - **closest_secondary_school** - Gives the name of the closest secondary school to the property.
 - **distance_to_closest_secondary** - Gives the direct distance between the property and the closest secondary school.
 - **secondary_school_ranking** - Gives the ranking of the closest secondary school to the property which is taken from xml file since scraping takes credentials which leads to security issues. If the school is not listed, append the value as "not ranked".
 
**2. Data Reshaping**
Here we started reshaping the data by using standardisatiion & transformation techniques on given attributes.

- **Z-Score Normalisation (standardisation)** - Using scikit library & the class we used is the StandardScaler class gives the scaled value.
- **Min-Max Normalization** - Using scikit library & the class we used is the MinMaxScaler class gives the scaled value.
- **Tranformation** - Used to compress the values in the dataframe for better prediction using 3 different techniques:

     **Root transformation** - Finding the square root value of the dataset & building the linear model from the obtained data.    
     **Square power transformation** - Finding the powered value of the dataset & building the linear model from the obtained data.  
     **Log transformation** - Finding the logarithmic value of the dataset & building the linear model from the obtained data.

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import os
import re
import geopandas as gpd
from shapely.geometry import Point,shape
from shapely.geometry.polygon import Polygon
from math import sin, cos, sqrt, radians, atan2
from datetime import datetime
import csv
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
import math
import ast
%matplotlib inline

##  1. Data Integration

**Reading the Group CSV file**

Reading the group csv file data using pandas and dropping duplicates in the data, inspecting the data using the head

In [2]:
group_data = pd.read_csv('Group007.csv') # Reading the csv file
group_data.drop_duplicates(inplace=True) # dropping the duplicates
group_data = group_data.reset_index(drop=True)
group_data.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,Landsize,Age,Lattitude,Longtitude
0,ID155633325,6 Shepherd St,3,h,2100000.0,4/06/2016,2.0,2.0,1255.0,49.0,-37.8322,145.1056
1,ID199093294,1/312 Hope St,1,u,351000.0,27/11/2016,1.0,1.0,0.0,69.0,-37.7614,144.9393
2,ID457288014,503/120 Greville St,3,u,1350000.0,12/11/2016,2.0,2.0,0.0,119.0,-37.8493,144.9908
3,ID903676173,11/781 Whitehorse Rd,2,u,520000.0,28/05/2016,1.0,1.0,0.0,14.0,-37.8161,145.1098
4,ID694412018,85 Montague St,3,h,700000.0,18/06/2016,1.0,0.0,199.0,72.0,-37.8299,144.9496


**Extracting Zip File contents to a folder**

Using ZipFile we unzip all the zip files using extract all command. we mention the names of the folder we want in a list, by changing the directory we unzip the folders, first we open a txt file in a write buffer mode and write the contents of the unzipped file to that txt file and close the file. Finally changing the directoy at the end.

In [3]:
os.mkdir('additional_data',mode=777) # Creating a folder
with ZipFile('gtfs.zip','r') as zipfile: # Opening  the zip file in read mode 
    zipfile.extractall('additional_data') # extracting the zip file contents
    
with ZipFile('vic_suburb_boundary.zip','r') as zipfile: # Opening  the zip file in read mode   
    zipfile.extractall()  # extracting the zip file contents

folder_list = ['1','2'] # Giving the folder names in a list
cwd_path = os.getcwd() # Getting the current working directory
os.chdir('additional_data/') # Changing the directory

for folder in folder_list:
    with ZipFile(folder+'\google_transit.zip','r') as inputfile: # Opening  the zip file in read mode 
        name_list = inputfile.namelist() # getting the names of the contents in the zip file
        name_list_new = [''.join(re.findall('(.*)\.txt',i)) for i in name_list] # Naming the txt files with folder name as extension
        for element in range(len(name_list)):
            output_file = open(name_list_new[element]+'_'+folder+'.txt','wb') # opening a new file in write buffer mode
            output_file.write(inputfile.read(name_list[element]))  # Writing the contents to the file
            output_file.close()
            
os.chdir(cwd_path) # Changing back the directory

**Reading the shp file**

Reading the shp file using geopandas and inspecting the data using head

In [4]:
shp_data = gpd.read_file('VIC_LOCALITY_POLYGON_shp.shp')
shp_data.head()

,LC_PLY_PID,DT_CREATE,DT_RETIRE,LOC_PID,VIC_LOCALI,VIC_LOCA_1,VIC_LOCA_2,VIC_LOCA_3,VIC_LOCA_4,VIC_LOCA_5,VIC_LOCA_6,VIC_LOCA_7,geometry
0,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"POLYGON ((141.74552399 -35.07228701, 141.74552..."
1,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"POLYGON ((148.668767 -37.39571245, 148.6687620..."
2,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"POLYGON ((142.92287999 -37.97885997, 142.90449..."
3,6673,2011-08-31,None,VIC734,2017-08-09,None,DEPTFORD,None,None,G,None,2,"POLYGON ((147.82335712 -37.66000897, 147.82312..."
4,6674,2011-08-31,None,VIC2900,2012-04-27,None,YANAC,None,None,G,None,2,"POLYGON ((141.279783 -35.99858911, 141.2798853..."


**Extracting Suburb**

Using Geometry function from geopandas we transform the shp data into a geometry. Using Point functions from shapely we transform the Lat and Lon to a geometry point. Now using a for loop over the geometry data we check whether a point is inside that geometry or not, if it is we extract the name of that geometry from that row and write it to the group data.

In [ ]:
polygon_data = shp_data.geometry.tolist() # Converting the geo pandas data frame column to a geometries 
lat_lon =[Point(xy) for xy in zip(group_data['Longtitude'], group_data['Lattitude'])] # Converting the lat and lon to Point
contain_list = []
for poly in polygon_data:
    sub_list = []
    for location in range(len(lat_lon)):
        if lat_lon[location].within(poly): # Checking if a Ploygon contains that point
            sub_list.append(location)
    contain_list.append(sub_list)
    
contain_ploy_dict = dict()
for row in range(len(contain_list)):
    if len(contain_list[row]) > 0:
        contain_ploy_dict[shp_data.loc[row,'VIC_LOCA_2']] = contain_list[row] # Writing the name of the suburb w.r.t to that point in a dict

for row in range(group_data.shape[0]):
    for key,values in contain_ploy_dict.items():
        if row in values:
            group_data.loc[row,'Suburb'] = key  # Writing the name of the suburb to the dataframe

**Train Station id, Distance to train station and Travel min to CBD**

Followingly we write functions which are used to extract the data from the txt files, we integrate the extracted content to the respective columns in group csv file.

We used only the folder 1 and 2 data as these are the only folders which have the data related to train. For the columns mentioned above we extracted content in the folders 1 and 2 to the folder, with respective folder name at the end of that of txt file. 

Reading the txt files using using pandas into dataframes and store it in a list. Using concat function in pandas we merge the dataframes and return the resultant dataframes.The input to the function is the name of the file.

In [ ]:
def merge_txt(match):
    '''
    @match: Name of the file you want to merge
    '''
    data = [] 
    file_list = [''.join(re.findall('(' + match + '_[1,2].txt)',file)) for file in os.listdir('additional_data')
                 if len(re.findall('(' + match + '_[1,2].txt)',file)) > 0] # Extracting the names of the file using regex 
    for file in file_list:
        data.append(pd.read_csv('additional_data\\'+file)) # Appending the dataframes to a list
    df = pd.concat(data, axis=0).reset_index(drop=True) # Concatinating the dataframes
    return df

agency_df = merge_txt('agency') 
calendar_df = merge_txt('calendar')
calendar_dates_df = merge_txt('calendar_dates')
routes_df = merge_txt('routes')
shapes_df = merge_txt('shapes')
stoptimes_df = merge_txt('stop_times')
stops_df = merge_txt('stops')
trips_df = merge_txt('trips')

This Function calculates the distance between two lat and lon points using haversine formula and returning the distance.

In [ ]:
def dist_lat_lon(house_lat,house_lon,train_lat,train_lon):
    '''
    @house_lat : The lattitude of the house
    @house_lon: The longititude of the house
    @train_lat: The lattitude of the train station
    @train_lon: The longititude of the train station
    '''
    R = 6378.0 # Radius of the earth
    house_lat, house_lon, train_lat, train_lon = map(radians, 
                                                     [house_lat, house_lon, train_lat, train_lon]) # Converting the lat and lon to radians
    
    lon_distance = house_lon - train_lon
    lat_distance = house_lat - train_lat
    
    dist1 = sin(lat_distance / 2)**2 + cos(house_lat) * cos(train_lat) * sin(lon_distance / 2)**2
    dist2 = 2 *atan2(sqrt(dist1), sqrt(1 - dist1))   
    distance = R * dist2
    return round(distance,4)*1000

Using regex on the departure_time to extract only hour and check if it's greater than 24, if it is then we exclude those rows from the dataframe. Using pd.to_datetime we convert the time column which is in string format to a time column

In [ ]:
stoptimes_df['Hour'] = stoptimes_df.apply(lambda x: list(map(int,re.findall('\d+',x['departure_time'])))[0],axis=1) # regex to extract hour from time
stoptimes_df = stoptimes_df[stoptimes_df['Hour'] < 24].reset_index(drop=True) # excluding those rows which have hour value greater than 24
stoptimes_df['departure_time'] = pd.to_datetime(stoptimes_df['departure_time'],format = '%H:%M:%S').dt.time # changing the type from string to datetime
stoptimes_df['arrival_time'] = pd.to_datetime(stoptimes_df['arrival_time'],format = '%H:%M:%S').dt.time # changing the type from string to datetime
stoptimes_df.drop(columns='Hour',axis=1,inplace=True)
stoptimes_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,7548.UR.1-V08-A-mjp-1.1.H,06:08:00,06:08:00,20291,1,NaN,0,0,0.000000
1,7548.UR.1-V08-A-mjp-1.1.H,06:42:00,06:42:00,20346,2,NaN,0,0,52720.840026
2,7548.UR.1-V08-A-mjp-1.1.H,06:55:00,06:57:00,20341,3,NaN,0,0,68786.649688
3,7548.UR.1-V08-A-mjp-1.1.H,07:16:00,07:16:00,20339,4,NaN,0,0,94514.728171
4,7548.UR.1-V08-A-mjp-1.1.H,07:32:00,07:34:00,20351,5,NaN,0,0,116895.210301


Filtering the calenders dataframe to extract the service id that operate only on weekdays, Using that service id's we extract the trip id's from the trips txt file and store it in a list after removing the duplicates.

In [ ]:
calendar_df['weekdays_only'] = calendar_df.apply(lambda x: 'Yes' if x['monday']+x['tuesday']+x['wednesday']+x['thursday']+x['friday'] == 5 
                                                 else 'No',axis=1)
# Using apply on the dataframe to check if the sum of all the weekday columns is 5 or not if it's we write yes else NO

serviceid_list = list(set(calendar_df[calendar_df['weekdays_only'] == 'Yes']['service_id'].tolist())) 
# taking the service id which has yes under weekday_only
print(serviceid_list)

trips_df['weekday_only'] = trips_df.apply(lambda x: 'Yes' if (x['service_id'] in serviceid_list) else 'No',axis=1) 
# Creating a new column and filling it with yes if the trip id of that row exists in the service id list

weekday_tripid_list = list(set(trips_df[trips_df['weekday_only'] == 'Yes']['trip_id'].tolist())) 
# Taking the trip ids from the row which has yes under weekday_only column

weekday_tripid_list[:3]

['T0_2', 'T0', 'T0_1', 'T0_3']


['9520.T0.2-PKM-F-mjp-1.6.H',
 '1539.T0.2-SYM-G-mjp-1.4.H',
 '2403.T0.2-SDM-F-mjp-1.1.H']

Getting the stopid's of the `Southern Cross Railway Station (Melbourne City)` from the stops_df and storing it in a list. Followingly we create a new dataframe which doesn't have the stopids mentioned in the list, and extract some specific columns from the stoptimes dataframe. Using `pd.Timestamp` we convert the string to timestamp and extract time from it by creating boundaries start time and end time. Now we apply some filters to to include only the trip id that are in the weekday_tripid_list and a time filter to include the rows that are within the time frame. Now we convert the time back to strings using str with apply.

In [ ]:
stop_id_list = list(set(stops_df[stops_df.stop_name == 'Southern Cross Railway Station (Melbourne City)']['stop_id'].tolist()))
# Extracting the stopid's that has Southern Cross Railway Station (Melbourne City) as the stopname

noscross_df = stoptimes_df[~stoptimes_df['stop_id'].isin(stop_id_list)][['trip_id','arrival_time','departure_time','stop_id','stop_sequence']] 
# Creating a new dataframe that doesn't have the stopid's that are in the list and selecting some specific column from the stoptimes dataframe

start_time = pd.Timestamp('07:00:00').time() # Converting a string to timestamp and extracting time from it
end_time = pd.Timestamp('09:30:00').time() # Converting a string to timestamp and extracting time from it

noscross_df = noscross_df[(noscross_df['trip_id'].isin(weekday_tripid_list))
                          &(noscross_df['departure_time'] >= start_time)&(noscross_df['departure_time'] <= end_time)].reset_index(drop=True)
# Using .isin we include the tripid's that are in the weekday_tripid_list and functioning within the time frame mentioned 

noscross_df['arrival_time'] = noscross_df.apply(lambda x: str(x['arrival_time']),axis=1) # Converting the time column back to string
noscross_df['departure_time'] = noscross_df.apply(lambda x: str(x['departure_time']),axis=1) # Converting the time column back to string
noscross_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,1051.T0.1-V08-G-mjp-1.1.H,07:16:00,07:16:00,20339,4
1,1051.T0.1-V08-G-mjp-1.1.H,07:32:00,07:34:00,20351,5
2,1051.T0.1-V08-G-mjp-1.1.H,07:45:00,07:45:00,20330,6
3,1051.T0.1-V08-G-mjp-1.1.H,07:57:00,07:57:00,20328,7
4,1051.T0.1-V08-G-mjp-1.1.H,08:16:00,08:16:00,20357,10


In this section we create a dataframe that only has the stopid's mentioned in the list and selecting some specific columns from it. Followingly we create a filter to include the tripids that are in the weekday_tripid_list. Now we convert the columns that are in the datetime format back to string and inspect the dataframe using head 

In [ ]:
scross_df = stoptimes_df[stoptimes_df['stop_id'].isin(stop_id_list)][['trip_id','arrival_time','departure_time','stop_id','stop_sequence']]
# Creating a new dataframe that only has the stop ids mentioned in the list and selecting some specific columns from the dataframe

scross_df =  scross_df[(scross_df['trip_id'].isin(weekday_tripid_list))].reset_index(drop=True)
# Creating a filter to include the tripid's that are in weekday_tripid_list

scross_df['arrival_time'] = scross_df.apply(lambda x: str(x['arrival_time']),axis=1) # Converting the time column back to string
scross_df['departure_time'] = scross_df.apply(lambda x: str(x['departure_time']),axis=1) # Converting the time column back to string
scross_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,1051.T0.1-V08-G-mjp-1.1.H,09:57:00,09:57:00,20043,23
1,1052.T0.1-V08-G-mjp-1.2.H,16:35:00,16:35:00,20043,23
2,1053.T0.1-V08-G-mjp-1.2.H,22:08:00,22:08:00,20043,23
3,1522.T0.1-V08-G-mjp-1.4.R,07:20:00,07:20:00,20043,1
4,1523.T0.1-V08-G-mjp-1.5.R,13:20:00,13:20:00,20043,1


Extracting the trip_id from the scross_df that only has the stop_id's of the southern cross station into a list and trip_id's of that doesn't have souhtern cross station stopid from the noscross_df into another list. Now we iterate over the noscross_list and with if condition we include that trip id which starts from any station between 07:00:00 and 09:30:00 and goes to southern cross directly and append these into a list. Now using this list we filter both of the dataframes scross_df and noscross_df to include the tripid's that are in the list.

In [ ]:
scross_list_notime = scross_df['trip_id'].tolist() # Extracting the trip_id to a list
noscross_list = noscross_df['trip_id'].tolist() # Extracting the trip_id to a list
new_scross_list_time = []

for i in noscross_list:
    if i in scross_list_notime:
        new_scross_list_time.append(i)

new_station_df1 = noscross_df[noscross_df['trip_id'].isin(new_scross_list_time)].reset_index(drop=True)
# Filtering the dataframe to include the trip_id that are in the list

new_scross_df = scross_df[scross_df['trip_id'].isin(new_scross_list_time)].reset_index(drop=True)
# Filtering the dataframe to include the trip_id that are in the list

Concatinating the dataframes using pd.concat. Using pd.to_timedelta we convert the time column which is in string format to the time format which is in minutes.

In [ ]:
final_df = pd.concat([new_station_df1,new_scross_df]).reset_index(drop=True)
# Concatinating the dataframes

final_df['departure_time'] = final_df.apply(lambda x: pd.to_timedelta(x['departure_time']).seconds/60,axis=1)
# Converting the time string column to time format using pd.to_timedelta

final_df['arrival_time'] = final_df.apply(lambda x: pd.to_timedelta(x['arrival_time']).seconds/60,axis=1)
# Converting the time string column to time format using pd.to_timedelta

final_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,1051.T0.1-V08-G-mjp-1.1.H,436.0,436.0,20339,4
1,1051.T0.1-V08-G-mjp-1.1.H,452.0,454.0,20351,5
2,1051.T0.1-V08-G-mjp-1.1.H,465.0,465.0,20330,6
3,1051.T0.1-V08-G-mjp-1.1.H,477.0,477.0,20328,7
4,1051.T0.1-V08-G-mjp-1.1.H,496.0,496.0,20357,10


To get the stop_id of the station that is closest to the house in the data, we compute distance between the house to all the stations using the lat and lon points and append that distance to a list. Using min function on the list we get the minimum distance in the list, Using index of that minimum distance we get the stopid from the stops_df. Now we extract all the trip_id from stoptimes_df using stop_id of the station into a list, we also extract the trip_id of the southern cross station. Using a for loop we iterate over station_trip_id and include those trip_id that are going to scross and store it in a list. If the length of the list is greater than zero then we write that stop_id and that distance to the dataframe or else if the length is not greater than zero we change the minimum distance and go for the next minimum.

In [ ]:
# Train station_id and distance

for row in range(group_data.shape[0]):
    min_dist = []
    lat1 = group_data.loc[row,'Lattitude'] # Lattitude of the house
    lon1 = group_data.loc[row,'Longtitude'] # Longtitude of the house
    for row2 in range(stops_df.shape[0]):
        lat2 = stops_df.loc[row2,'stop_lat'] # Lattitude of the station 
        lon2 = stops_df.loc[row2,'stop_lon'] # Longtitude of the station
        min_dist.append(dist_lat_lon(lat1,lon1,lat2,lon2)) # Passing the inputs to the function
    e = True    
    while e:
        min_dist_value = min(min_dist) # getting the minimum distance from the list
        stop_id = stops_df.loc[min_dist.index(min_dist_value),'stop_id'] # Extracting the stop_id at the index of the minimum distance
        station_trip_id = final_df[final_df['stop_id'] == stop_id]['trip_id'].tolist() # Extracting the trip_id's for the stop_id
        scross_trips = final_df[final_df['stop_id'].isin(stop_id_list)]['trip_id'].tolist() # Extracting the trip_id's for the southern cross
        station_to_scross = [k for k in station_trip_id if k in scross_trips] # Include the trip_id that goes to the southern cross
        if len(station_to_scross)>0:
            group_data.loc[row,'distance_to_train_station'] = round(min_dist_value,3) # Write the minimum distance to the dataframe
            group_data.loc[row,'train_station_id'] = stop_id # write the stop_id to the dataframe
            e = False
        else:
            min_dist[min_dist.index(min_dist_value)] = min_dist_value+3 # Changing the minimum distance

To get the minimum travel time to CBD, we get the all the stop_id from the dataframe final_df and applying set on it to eliminate duplicates and convert it to a list. Now we use a for loop to iterate over the final_stopid_list, using that stop_id we extract all the trip_id's of that stop_id into a list. Using a for loop we iterate over the trip_list, using a trip_id and stop_id we get the station_deptime and using scross stopid and trip_id from the for loop we get the arrival time. If the length of these lists are not equal to zero, then we subract the scross_arr_time - station_dep_time and get the absolute value from it. If the length of the time_diff list is greater than zero we get the avg from the list else we get the sum from it. Finally we write the dictonary value of that stop_id key to the dataframe.  

In [ ]:
# travel_min_to_CBD

final_stopid_list = list(set(final_df['stop_id'].tolist())) # Getting the stop_id from the dataframe
stop_avgtime_dict = dict()

for stop_id in final_stopid_list:
    time_diff = []
    trip_list = final_df[final_df['stop_id'] == stop_id]['trip_id'].tolist() # trip_id's of that stop_id
    for trip in trip_list:
        station_deptime  = final_df[(final_df['trip_id'] == trip)&(final_df['stop_id'] == stop_id)]['departure_time'].tolist() # dep_time of a station which has that stop_id and trip_id from the loop
        scross_arrtime   = final_df[(final_df['trip_id'] == trip)&(final_df['stop_id'].isin(stop_id_list))]['arrival_time'].tolist() # arr_time of the scross_station which has the same trip_id that has used above
        if len(station_deptime) != 0 and len(scross_arrtime) != 0: 
            time_diff.append(abs(scross_arrtime[0]-station_deptime[0])) # Subracting the station_deptime from the scross_arrtime
    if len(time_diff)>0:
        stop_avgtime_dict[stop_id] = round(sum(time_diff)/len(time_diff),2) # Average value from the list
    else:
        stop_avgtime_dict[stop_id] = sum(time_diff)
        
group_data['travel_min_to_CBD'] = group_data.apply(lambda x: stop_avgtime_dict[x['train_station_id']],axis=1) # Using dictionary to get the value from the stop_id key
group_data.head()

**Over Priced**

We need to check whether the property price is higher than the median of properties with similar contents w.r.t. `'Rooms','Bathroom','Car','Type','Suburb','Year'`.

First we had extarcted the year from Date column using `pd.to_datetime`. Then calculated the median of the property for similar characteristics i.e., `'Rooms','Bathroom','Car','Type','Suburb','Year'` w.r.t. `Price`. Then checking price of the property is higher than its median price & appending as True or else as  False to a new column. 

In [ ]:
# Extracting the year from Date column
group_data['Year'] = pd.to_datetime(group_data['Date']).dt.year
# Calculating the median of the property for similar characteristics
df = group_data.groupby(['Rooms','Bathroom','Car','Type','Suburb','Year'])['Price'].median().reset_index()
# Merging the grouped contents with the main dataset
df2 = pd.merge(group_data,df,on=['Rooms','Bathroom','Car','Type','Suburb','Year']).reset_index()
# Checking price of the property is higher than its median price & appending as True or else as False to a new column
df2['over_priced?'] = df2.apply(lambda x: x['Price_x'] > x['Price_y'],axis=1)
# Finally merging the contents of new column to main dataframe w.r.t `ID`
group_data = pd.merge(group_data,df2[['ID','over_priced?']],on='ID')
group_data.head()

**Columns `crime_A_average`, `crime_B_average`, `crime_C_average`**

Before calculating the crime average, first with the help of `councils.txt` we need to extract the council names w.r.t its suburbs to our dataframe. First I had created an empty dictionary and splitted the each line in data file with `: `. Using `ast.literal_eval` library I had extracted the values which are suburb names by converting it to upper case to match with our data frame and store it in the dictionary.

After extracting suburbs & council names from the data file, I had appended the council name for its respective suburb names.

In [ ]:
crime_dict= dict()
#Reading councils.txt file
with open('councils.txt') as input_file:
    for line in input_file:
        (key,value) = line.split(": ")
# Extracted the values which are suburb names by converting it to upper case to match with our data frame and store it in the dictionary.
        crime_dict[key.strip()] = [i.strip().upper() for i in ast.literal_eval(value)]
# Iterating through group data
for row in range(group_data.shape[0]):
# Iterating for key, values found in dictionary
    for key,value in crime_dict.items():
        if group_data.loc[row,'Suburb'] in value:
            group_data.loc[row,'Council'] = key # Appending the council names to its respective suburbs
            
group_data.head()

Reading the excel file and filtering the excel contents with `sheet_name` & skipping few rows i.e., to remove empty rows.

Then dropping the null column from the dataframe & viewing the dataframe contents.

In [ ]:
# Reading excel file as input with adding few options like sheet_name & skiprows to remove empty rows.
crime_df = pd.read_excel("crimebylocationdatatable-yearending31march2016.xlsx",sheet_name='Table 1',skiprows=19)
crime_df = crime_df.dropna(1, how = 'all')
crime_df.head()

Here we need to calculate the average value for three different types of crimes i.e.,`type-A`, `type-B`, `type-C`. We had defined a function which takes two arguments i.e., type of crime & year, council in main dataframe.

First we had extracted the year list for the last 3 years. Then filtered the crime_type w.r.t council, year, offence count and taken the sum of all the offences that occurred in each year & in each council and dividing the sum of all offence counts by 3. Finally we rounded the average value by 3.

Then applying lambda function to add extracted crime data for each type of offence to three different columns.

In [ ]:
'''
Function to fetch the average value for different types of crimes.
'''
def crime_avg_func(crime_csa,row):
    '''
    @crime_csa: Type of crime to calculate average
    @row: Checking the year & council in the dataframe
    '''
    year = row['Year']
    council = row['Council']
# Extracting the year list for last 3 years
    year_list = [int(year)-1,int(year)-2,int(year)-3]
# Filtering the type of crime w.r.t. council, year & offence count & stored it in a dataframe
    df = crime_df[crime_df['CSA Offence Division']==crime_csa][['Local Government Area','Apr - Mar reference period','Offence Count']]
# Taking the sum of offence count by grouping council & year
    df = df.groupby(['Local Government Area','Apr - Mar reference period'])['Offence Count'].sum().reset_index()
# Checking the average by dividing offence count sum by 3  
    avg_value = df[(df['Apr - Mar reference period'].isin(year_list)) & (df['Local Government Area']==council)]['Offence Count'].sum()/3
    return round(avg_value,3)
# Applying lambda function to add extracted crime data for each type of offence to three different columns
group_data['crime_A_average'] = group_data.apply(lambda x: crime_avg_func('A Crimes against the person',x),axis=1)
group_data['crime_B_average'] = group_data.apply(lambda x: crime_avg_func('B Property and deception offences',x),axis=1)
group_data['crime_C_average'] = group_data.apply(lambda x: crime_avg_func('C Drug offences',x),axis=1)
group_data.head()

**Web Scraping of website 1**

From the given url, we had read the url contents with the help of `urlopen` library. By using `BeautifulSoup` library, we had parsed our url contents and stored it in a variable. Contents of the url are to be written in a csv file for further accessing it as a dataframe. 

Since we need to extract the table contents, with the help of beautiful soup, first we had found the `table`. Then we observed that the column header are stored in the `tr` (table row) HTML tag. First we had iterated through the column names found in `tr` tag. Then we had to extract the column contents like school name, ranking etc which is present under HTML tags `th` & `td` (table header & table data cell). This id done by using `findAll` option where we got all the column contents by iterating through a loop and appending the contents found with `.text` row-wise.

Finally after extarcting all the table contents we had written it to a csv file for easier access.

In [ ]:
# Reading the website url
html = urlopen("http://www.schoolcatchment.com.au/?p=12301").read()
# Parsing url using BeautifulSoup
bsobj = BeautifulSoup(html, 'html.parser')
filename = "primary_school_data.csv"
# Crating a csv to write the website contents
csv_writer = csv.writer(open(filename, 'w',newline=''))
# Finding the table from the beautifulsoup object
head = bsobj.find('table')
rows=[]
# Extracting the column name contents in a table by iterating through for loop
for value in head.findAll('tr'):
    data = []
# Extarcting the table contents & appending it to a list by iteraring through for loop
    for value1 in value.findAll(['th','td']):
        data.append(value1.text)
    rows.append(data)
# Writing the extracted contents into a csv file.
for data1 in rows:
    csv_writer.writerow(data1)

In [ ]:
# Reading the extarcted csv file using pandas
website1 = pd.read_csv('primary_school_data.csv')
# Dropping the duplicate school names by keeping its first occurence & resetting the index.
website1 = website1.drop_duplicates(["School Name"], keep='first').reset_index(drop=True)
website1.shape

**Columns `closest_primary_school`, `distance_to_closest_primary` & `primary_school_ranking`**

With the help of the input `schools.xml` file I had extracted School names, School type, longitude & latitude using `Schools.xml` file. This is achieved with the help of regular expressions to the data file. Initially using `re.findall`, I had extracted the contents inside the `<school>(.*?)</school>` & `</(.*?)>` patterns to extract the information of each school. Then to extract the full data in the file, we had iterated it row-wise & extracted the columns in a list. 

Using `re.sub` I had replaced few special characters like `&apos;` & `&amp;` with `'` and `&`. Then converting the obtained list to a dataframe to view the contents of school.

In [ ]:
# Reading the contents of xml file
with open('schools.xml','r') as inputfile:
    schools_list = inputfile.readlines()
    
schools_data = ''.join(schools_list)
# Removing any observed empty spaces in the data
schools_data = re.sub('\n','',schools_data)
# Extracting the chunk data of school
row_list = [i.strip() for i in re.findall(r'<school>(.*?)</school>',schools_data)]
column_names = re.findall('</(.*?)>',row_list[0])
full_data = []

# Iterating the chunk data row-wise using for loop
for row in row_list:
    data_list = re.split('<.*?>',row)
    data_list = [i for i in data_list if i != '    ']
    data_list = data_list[1:-1]
    data_list = [re.sub('&amp;','&',i) for i in data_list]
    data_list = [re.sub('&apos;','\'',i) for i in data_list]
    full_data.append(data_list)
# Creating a dataframe of column names
school_df = pd.DataFrame(columns=column_names)
# Appending the values in a dataframe by iterating through for loop
for row in range(len(full_data)):
    school_df = school_df.append(pd.Series(full_data[row],index=column_names),ignore_index=True)

school_df.head()

Since we are used to find the closest primary school from the property & shortest distance we had filtered the `school_type` to either `Primary` or `Pri/Sec` to get data of primary school alone.

Filtering the contents in a separate list and extracting the `School_name`, `Longitude` & `Latitude`.

In [ ]:
# Creating a list to filter the contents i.e., Primary & Pri/Sec
school_type_list = ['Primary','Pri/Sec']
# Extracting the school name, Longitude & Latitude from the filtered list
school_name_list = school_df.apply(lambda x: (x['School_Name'],x['X'],x['Y']) if x['School_Type'] in school_type_list else 'NA' ,axis=1).tolist()
school_name_list = [i for i in school_name_list if i != 'NA']

To find the nearest school from property & its minimum distance we had created a for loop to iterate through group data, first we extracted the property latitude & longitude separately. Then gain iterating the filtered school list from above we extracted the school's latitude & longitude separately. With the help of `haversine` formula we had calculated the distance between school & property. Then taken the minimum distance using `min` function. From the school_distance variable where we had appended the school name as well as its shortest distance. 

Finally writing those two contents into a dataframe of two new columns.

In [ ]:
for row in range(group_data.shape[0]):
    school_distance = []
    lat1 = group_data.loc[row,'Lattitude'] # Extracting latitude of property
    lon1 = group_data.loc[row,'Longtitude'] # Extracting latitude of property
    for row2 in range(len(school_name_list)):
        lat2 = float(school_name_list[row2][2]) # Extracting latitude of school
        lon2 = float(school_name_list[row2][1]) # Extracting latitude of school
# Calculating the shortest distance between two lat & lon using Haversine formula
        school_distance.append(dist_lat_lon(lat1,lon1,lat2,lon2))
# Finding the minimum distance
    minimum_distance_school = min(school_distance)
# Creating a new columns & appending values of school names & its shortest distance
    group_data.loc[row,'closest_primary_school'] = school_name_list[school_distance.index(minimum_distance_school)][0]
    group_data.loc[row,'distance_to_closest_primary'] = minimum_distance_school

Finally we are given to extract the rankings of the primary schools from the scraped data above to the schools obtained in our dataframe. Extracting a list of school names & its rankings to a separate list. List has two indexes with school name in first index & rank in second index. Using `apply` function, checking if school name in `closest_primary_school` matches with the extracted school name list we appended its respective rank to the dataframe in new column called `primary_school_ranking`, if not we appended the column with `not ranked`.

In [ ]:
# Extracting a list of school names & its rankings to a separate list
primary_school_rank_list = website1.apply(lambda x: (x['School Name'],x['Ranking']),axis=1)
# Extracting school name in first index
primary_school = [k[0] for k in primary_school_rank_list]
# Extracting school rank in first index
primary_rank = [k[1] for k in primary_school_rank_list]
# Appending the ranks of schools with its ranks from the data obtained
group_data['primary_school_ranking'] = group_data.apply(lambda x: primary_rank[primary_school.index(x['closest_primary_school'])] 
                                         if x['closest_primary_school'] in primary_school else 'not ranked',axis=1 )
group_data.head()

**Secondary School**

Secondary school rankings is present in website but to access taht we need to provide credentials which is not a safer way. Hence we had taken the source code & stored it as `.xml` file.

With the help of the input `website2.xml` file I had extracted School names, School rank etc using `website2.xml` file. This is achieved with the help of regular expressions to the data file. Initially using `re.findall`, I had extracted the contents inside the `<div style="display:none">(.*?)</div>` & `<li>(.*?)</li>` patterns to extract the information of each school. Then to extract the full data in the file, we had iterated it row-wise & extracted the columns in a list. While appending the values again converting the list to series using `re.findall`, I had extarcted the `<div>(.*?)</div>` contents indiviually & then converting the obtained list to a dataframe to view the contents of school.

Dropping the duplicate school names by keeping its first occurence & resetting the index.

In [ ]:
# Reading the contents of xml file
with open('website2.xml','r') as inputfile:
    website2_list = inputfile.readlines()
    
website2_data = ''.join(website2_list)
# Removing any observed empty spaces in the data
website2_data = re.sub('\n','',website2_data)
# Extracting the chunk data of school
column_names = [i.strip() for i in re.findall(r'<div style="display:none">(.*?)</div>',website2_data)]
# Extracting the contents of the data
data_row_list = re.findall('<li>(.*?)</li>',website2_data)

website2_df = pd.DataFrame(columns=column_names)
# Iterating the chunk data row-wise using for loop
for row in range(len(data_row_list)):
    website2_df = website2_df.append(pd.Series(re.findall('<div>(.*?)</div>',data_row_list[row]), index=column_names),ignore_index=True)
# Dropping the duplicate school names by keeping its first occurence & resetting the index.    
website2_df = website2_df.drop_duplicates(["School Name"], keep='first').reset_index(drop=True)
website2_df.head()

Since we are used to find the closest primary school from the property & shortest distance we had filtered the `school_type` to either `Secondary` or `Pri/Sec` to get data of primary school alone.

Filtering the contents in a separate list and extracting the `School_name`, `Longitude` & `Latitude`.

In [ ]:
# Creating a list to filter the contents i.e., Secondary & Pri/Sec
sec_school_type_list = ['Secondary','Pri/Sec']
# Extracting the school name, Longitude & Latitude from the filtered list
sec_school_name_list = school_df.apply(lambda x: (x['School_Name'],x['X'],x['Y']) if x['School_Type'] in sec_school_type_list else 'NA' ,axis=1).tolist()
sec_school_name_list = [i for i in sec_school_name_list if i != 'NA']

To find the nearest school from property & its minimum distance we had created a for loop to iterate through group data, first we extracted the property latitude & longitude separately. Then gain iterating the filtered school list from above we extracted the school's latitude & longitude separately. With the help of `haversine` formula we had calculated the distance between school & property. Then taken the minimum distance using `min` function. From the school_distance variable where we had appended the school name as well as its shortest distance. 

Finally writing those two contents into a dataframe of two new columns.

In [ ]:
for row in range(group_data.shape[0]):
    school_distance = []
    lat1 = group_data.loc[row,'Lattitude']# Extracting latitude of property
    lon1 = group_data.loc[row,'Longtitude']# Extracting longitude of property
    for row2 in range(len(sec_school_name_list)):
        lat2 = float(sec_school_name_list[row2][2])# Extracting latitude of school
        lon2 = float(sec_school_name_list[row2][1])# Extracting longitude of school
# Calculating the shortest distance between two lat & lon using Haversine formula
        school_distance.append(dist_lat_lon(lat1,lon1,lat2,lon2))
# Finding the minimum distance
    minimum_distance_school = min(school_distance)
# Creating a new columns & appending values of school names & its shortest distance
    group_data.loc[row,'closest_secondary_school'] = sec_school_name_list[school_distance.index(minimum_distance_school)][0]
    group_data.loc[row,'distance_to_closestsecondary'] = minimum_distance_school

Finally we are given to extract the rankings of the primary schools from the scraped data above to the schools obtained in our dataframe. Extracting a list of school names & its rankings to a separate list. List has two indexes with school name in first index & rank in second index. Using `apply` function, checking if school name in `closest_secondary_school` matches with the extracted school name list we appended its respective rank to the dataframe in new column called `secondary_school_ranking`, if not we appended the column with `not ranked`.

In [ ]:
# Extracting a list of school names & its rankings to a separate list
sec_school_rank_list = website2_df.apply(lambda x: (x['School Name'],x['Ranking']),axis=1)
# Extracting school name in first index
sec_school = [k[0] for k in sec_school_rank_list]
# Extracting school rank in first index
sec_rank = [k[1] for k in sec_school_rank_list]
# Appending the ranks of schools with its ranks from the data obtained
group_data['secondary_school_ranking'] = group_data.apply(lambda x: sec_rank[sec_school.index(x['closest_secondary_school'])] 
                                         if x['closest_secondary_school'] in sec_school else 'not ranked',axis=1 )
group_data.head()

Reordering & renaming the column names as per the given specifications file & saving the output in `.CSV` format.

In [ ]:
group_data = group_data[['ID','Address','Suburb','Price','Type','Date','Rooms','Bathroom','Car','Landsize','Age',
                         'Lattitude','Longtitude','train_station_id','distance_to_train_station','travel_min_to_CBD',
                         'over_priced?','crime_A_average','crime_B_average','crime_C_average','closest_primary_school',
                         'distance_to_closest_primary','primary_school_ranking','closest_secondary_school',
                         'distance_to_closestsecondary','secondary_school_ranking']]


group_data = group_data.rename(columns = {'Lattitude':'Latitude','Longtitude':'Longitude','Landsize':'LandSize'})
group_data.to_csv('Group007_solution.csv',index=False)
group_data.head()

## Task 2: Data Reshaping

When the results of the datasets are affecting too much in their results, a good level prediction is provided by normalisation or scaling.

Reshaping the obtained dataframe with the required columns `Rooms`,`crime_C_average`, `travel_min_to_CBD` and `property_age` by building a model on `price` attributes to study the effect of different normalization/transformation methods i.e., `standardization`, `min-max normalization`, `log`, `power`, and `root transformation`.

In [ ]:
# Re-shaping the given grouped data with required columns
reshaping_df = group_data[['Rooms','crime_C_average','travel_min_to_CBD','Age','Price']]
# Ignoring the 0 value in travel_min_to_CBD column
reshaping_df = reshaping_df[reshaping_df['travel_min_to_CBD'] != 0].reset_index(drop=True)
reshaping_df.head()

In [ ]:
# Creating subplots to visualise the data before normalisation
fig = plt.figure(figsize=[15,5])

ax1 = fig.add_subplot(221)
ax1.hist(reshaping_df['Rooms'])
plt.title('Before performing normalisation/transformation on Rooms')

ax2 = fig.add_subplot(222)
ax2.hist(reshaping_df['crime_C_average'])
plt.title('Before performing normalisation/transformation on Crimes')

ax3 = fig.add_subplot(223)
ax3.hist(reshaping_df['travel_min_to_CBD'])
plt.title('Before performing normalisation/transformation on travel min to CBD')

ax4 = fig.add_subplot(224)
ax4.hist(reshaping_df['Age'])
plt.title('Before performing normalisation/transformation on Age')

plt.tight_layout()

From the histogram plots we can see that the data given is right skewed.

#### 1. Z-Score Normalisation (standardisation): 

Here we are going to use scikit library & the class we used is the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html">StandardScaler</a> class. 

The task here it to standardise the values of Rooms, crime_C_average, travel_min_to_CBD and Age and append the standard variables to the DataFrame `df_std`.

In [ ]:
# Performing normalisation on reshaping dataframe
std_scale = preprocessing.StandardScaler().fit(reshaping_df[['Rooms','crime_C_average','travel_min_to_CBD','Age']])
# Applying transform function to the scaled data
df_std = std_scale.transform(reshaping_df[['Rooms','crime_C_average','travel_min_to_CBD','Age']])
df_std[0:5]

In [ ]:
# Creating subplots to visualise the data before standard normalisation & after standard normalisation

# Adding the new columns for the scaled data.
reshaping_df['Room_scaled'] = df_std[:,0] 
reshaping_df['crime_C_avg_scaled'] = df_std[:,1]
reshaping_df['travel_min_to_CBD_scaled'] = df_std[:,2]
reshaping_df['Age_scaled'] = df_std[:,3]

fig = plt.figure(figsize=[15,5])

ax1 = fig.add_subplot(221)
reshaping_df['Rooms'].plot(ax=ax1)
reshaping_df['Room_scaled'].plot(ax=ax1);
plt.legend()
plt.title('Rooms in the Data')

ax2 = fig.add_subplot(222)
reshaping_df['crime_C_average'].plot(ax=ax2)
reshaping_df['crime_C_avg_scaled'].plot(ax=ax2)
plt.title('Crimes in the data')
plt.legend()


ax3 = fig.add_subplot(223)
reshaping_df['travel_min_to_CBD'].plot(ax=ax3)
reshaping_df['travel_min_to_CBD_scaled'].plot(ax=ax3)
plt.title('travel to CBD in the data')
plt.legend()

ax4 = fig.add_subplot(224)
reshaping_df['Age'].plot(ax=ax4)
reshaping_df['Age_scaled'].plot(ax=ax4)
plt.title('Age of the house in the data')
plt.legend()
plt.tight_layout()

In [ ]:
# Creating subplots to visualise the data after min-max normalisation
fig = plt.figure(figsize=[15,5])

ax1 = fig.add_subplot(221)
ax1.hist(reshaping_df['Room_scaled'])
plt.title('After using Z-Score on Rooms')

ax2 = fig.add_subplot(222)
ax2.hist(reshaping_df['crime_C_avg_scaled'])
plt.title('After using Z-Score on Crimes')

ax3 = fig.add_subplot(223)
ax3.hist(reshaping_df['travel_min_to_CBD_scaled'])
plt.title('After using Z-Score on travel min to CBD')

ax4 = fig.add_subplot(224)
ax4.hist(reshaping_df['Age_scaled'])
plt.title('After using Z-Score on Age')

plt.tight_layout()

If we take a look at the graphs above, we can see that the features are standardized to zero mean. As a Z score means both the mean and variability of data, We can not say that the Z-score is always better because all the times it assumes the distribution as normal.

### 2. Min-Max Normalisation:

Here we are going to use scikit library & the class we used is the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html">MinMaxScaler</a> class. 

The task here it to standardise the values of Rooms, crime_C_average, travel_min_to_CBD and Age and append the standard variables to the DataFrame `df_std`.

In [ ]:
# Performing normalisation on reshaping dataframe
minmax_scale = preprocessing.MinMaxScaler().fit(reshaping_df[['Rooms','crime_C_average','travel_min_to_CBD','Age']])
# Applying transform function to the scaled data
df_minmax = minmax_scale.transform(reshaping_df[['Rooms','crime_C_average','travel_min_to_CBD','Age']])
df_minmax[0:5]

In [ ]:
# Adding the new columns for the scaled data.
reshaping_df['Room_minmax'] = df_minmax[:,0] 
reshaping_df['crime_C_avg_minmax'] = df_minmax[:,1]
reshaping_df['travel_min_to_CBD_minmax'] = df_minmax[:,2]
reshaping_df['Age_minmax'] = df_minmax[:,3]

In [ ]:
# Creating subplots to visualise the data after min-max normalisation
fig = plt.figure(figsize=[15,5])

ax1 = fig.add_subplot(221)
ax1.hist(reshaping_df['Room_minmax'])
plt.title('After using Minmax on Rooms')

ax2 = fig.add_subplot(222)
ax2.hist(reshaping_df['crime_C_avg_minmax'])
plt.title('After using Minmax on Crimes')

ax3 = fig.add_subplot(223)
ax3.hist(reshaping_df['travel_min_to_CBD_minmax'])
plt.title('After using Minmax on travel min to CBD')

ax4 = fig.add_subplot(224)
ax4.hist(reshaping_df['Age_minmax'])
plt.title('After using Minmax on Age')

plt.tight_layout()

If you take a look at the graphs after scaling with MinMax,  It is very similar to the Z-Score Normalisation but the data is right skewed and the range is between 0 and 1. This is much better when you are creating a linear model, The only disadvantage in this model is this will underfit the data as it is baised towards one side comapared to other.

## 3. Data Transformation:

Another way to reshape data is to perform data transformation. Here we have to compress our values in the dataset depending on the method which we are performing. 

Here we had performd the following three kinds of transformations
* Root transformation
* Square power transformation
* Log transformation

### Root transformation:

Applying `sqrt` function from `math` library on the reshaped columns to find the square root value of the dataset & building the linear model from the obtained data.

In [ ]:
# Creating new columns on squared root value of data
for index,row in reshaping_df.iterrows():
    reshaping_df.loc[index, 'price_sqrt'] = sqrt(row['Price'])
    reshaping_df.loc[index, 'rooms_sqrt'] = sqrt(row['Rooms'])
    reshaping_df.loc[index, 'crime_C_average_sqrt'] = sqrt(row['crime_C_average'])
    reshaping_df.loc[index,'travel_min_to_CBD_sqrt'] = sqrt(row['travel_min_to_CBD'])
    reshaping_df.loc[index, 'age_sqrt'] = sqrt(row['Age'])

In [ ]:
# Creating subplots to visualise the data after Root transformation
fig = plt.figure(figsize=[15,5])

ax1 = fig.add_subplot(221)
ax1.hist(reshaping_df['rooms_sqrt'])
plt.title('After using Square root transformation on Rooms')

ax2 = fig.add_subplot(222)
ax2.hist(reshaping_df['crime_C_average_sqrt'])
plt.title('After using Square root transformation on Crimes')

ax3 = fig.add_subplot(223)
ax3.hist(reshaping_df['travel_min_to_CBD_sqrt'])
plt.title('After using Square root transformation on travel min to CBD')

ax4 = fig.add_subplot(224)
ax4.hist(reshaping_df['age_sqrt'])
plt.title('After using Square root transformation on Age')

plt.tight_layout()

If you take a look at the graphs above, we can see that the root transformation is very similar to the original data but with less variance, As the variance in the data is reduced this transformation will improve the accuracy of the predicition by the model.

### Square power transformation:

Applying `pow` function from `math` library on the reshaped columns to find the powered value of the dataset & building the linear model from the obtained data.

In [ ]:
# Creating new columns on power value of data
for index,row in reshaping_df.iterrows():
    reshaping_df.loc[index, 'price_pwr'] = pow(row['Price'],2)
    reshaping_df.loc[index, 'rooms_pwr'] = pow(row['Rooms'],2)
    reshaping_df.loc[index, 'crime_C_average_pwr'] = pow(row['crime_C_average'],2)
    reshaping_df.loc[index,'travel_min_to_CBD_pwr'] = pow(row['travel_min_to_CBD'],2)
    reshaping_df.loc[index, 'age_pwr'] = pow(row['Age'],2)

In [ ]:
# Creating subplots to visualise the data after square power transformation
fig = plt.figure(figsize=[15,5])

ax1 = fig.add_subplot(221)
ax1.hist(reshaping_df['rooms_pwr'])
plt.title('After using square power transformation on Rooms')

ax2 = fig.add_subplot(222)
ax2.hist(reshaping_df['crime_C_average_pwr'])
plt.title('After using square power transformation on Crimes')

ax3 = fig.add_subplot(223)
ax3.hist(reshaping_df['travel_min_to_CBD_pwr'])
plt.title('After using square power transformation on travel min to CBD')

ax4 = fig.add_subplot(224)
ax4.hist(reshaping_df['age_pwr'])
plt.title('After using square power transformation on Age')

plt.tight_layout()

If you take a look at the graphs above, we can see that the square power transformation increased the variance in the given data, As the variance in the data is increased and skewed towards right this transformation will reduce the accuracy of the predicition by the model compared to the original.

### Log transformation:

Applying `log` function from `math` library on the reshaped columns to find the logarithmic value of the dataset & building the linear model from the obtained data.

In [ ]:
# Creating new columns on log value of data
for index,row in reshaping_df.iterrows():
    reshaping_df.loc[index, 'price_log'] = math.log(row['Price'])
    reshaping_df.loc[index, 'rooms_log'] = math.log(row['Rooms'])
    reshaping_df.loc[index, 'crime_C_average_log'] = math.log(row['crime_C_average'])
    reshaping_df.loc[index, 'age_log'] = math.log(row['Age'])
    reshaping_df.loc[index,'travel_min_to_CBD_log'] = math.log(row['travel_min_to_CBD'])

In [ ]:
# Creating subplots to visualise the data after log transformation
fig = plt.figure(figsize=[15,5])

ax1 = fig.add_subplot(221)
ax1.hist(reshaping_df['rooms_log'])
plt.title('After using log transformation on Rooms')

ax2 = fig.add_subplot(222)
ax2.hist(reshaping_df['crime_C_average_log'])
plt.title('After using log transformation on Crimes')

ax3 = fig.add_subplot(223)
ax3.hist(reshaping_df['travel_min_to_CBD_log'])
plt.title('After using log transformation on travel min to CBD')

ax4 = fig.add_subplot(224)
ax4.hist(reshaping_df['age_log'])
plt.title('After using log transformation on Age')

plt.tight_layout()

As the data is highly skewed, by applying the log transformation will decrease the skewness in the data. As after this the data will be more useful for interpretation comapred to the originall data, which will help in predicting better results compared to others.

**Coming to conclusion from the histograms we plotted above**

Generally, A linear model assumes that the relationship between the variables in the dataset which are passed to it is linear, and the residuals that are fitted to the model assumes it as a normal distribution and the residuals size is almost same along your line that is fitted to the data. When the data given to us don't follow these assumptions a transformation on the data can improve the data.  

1. Normalization: By doing Normalization you might loose some valuable information in the data in the process and this works much better if the data given is normalized. A Normalization typically rescale the values.

    - Z- Score: After normalising the data with z-score, if we take a look at the data by plotting histogram. It is very similar to the original data and the data is still skewed after this step, which says that this doesn't change the data much. So if you build a linear model with the data you might still get the same result.

    - Min Max: After normalising the data with MinMax, the whole data is scaled between 0 and 1, this will not handel the outliers very well. So we might loose some data. From the histogram we can see that the skeweness is almost same as the Original data.
    
2. Transformation: When the data is not normalized transforming data may improve the predicitability or the fit for the general linear model

    - Square-root: Apply square root transformation on the data will decrease the variance in the data, by reducing the residual size towards the fitted line. So applying square root will minimize the large residuals size. So square root is one of the preferred method on the non-normalize data. From the histogram we can see that the skeweness in the data in reduced compared to others.

    - Squared: Squaring the data is not recommended as it increases the variance in the given data and size of the large residuals gradually increase compared to others, so this method is not recommended over others. The Skewness in the data is increased compared to the original data.

    - Log: Amongst all the different transformations, the most effective transformation is `Logarithmic transformation` which makes a highly right skewed dataset. During modelling with non-linear relationships, the chances of making mistakes can also be negatively skewed. In theory, when making a prediction, we want to make the smallest possible error, while also considering that we should not overfit the model. Overfitting occurs when too many dependent variables are in existence that the dataset is not generalized enough to make a valid prediction. From the histogram we can see that the skewness is drastically reduced, so this is much preferred compared to others.

So from the graphs above for each transformation we can say that Square-root or Log transformation may be one of the suited methods for the given data. To get more insight we can build a linear model for each transformation and see which one is better compared to others.

**Building linear model to test our theory and to confirm which transformation is better for the data.**

In [ ]:

# Linear Model on the Original data
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(reshaping_df[
    ['Rooms','crime_C_average','travel_min_to_CBD','Age']],reshaping_df['Price'],random_state=100)
# Developing LinearRegression model
lr_model_1 = LinearRegression()
# Fitting the model into our trained data
lr_model_1.fit(X_train_lr,y_train_lr)
accuracy_dict = dict()
# Calculating R-Squared value
accuracy_dict['Original without scaling'] = round(lr_model_1.score(X_test_lr,y_test_lr)*100,2)
print ('R-squared for the data without scaling using LinearRegression model = ',accuracy_dict['Original without scaling'],'%')


# Z-Score Normalisation on the data
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(reshaping_df[
    ['Room_scaled','crime_C_avg_scaled','travel_min_to_CBD_scaled','Age_scaled']],reshaping_df['Price'],random_state=100)
# Developing LinearRegression model
lr_model_2 = LinearRegression()
# Fitting the model into our trained data
lr_model_2.fit(X_train_lr,y_train_lr)
# Calculating R-Squared value
accuracy_dict['Z-Score'] = round(lr_model_2.score(X_test_lr,y_test_lr)*100,2)
print ('R-squared for the data after scaling with Normalisation using LinearRegression model',accuracy_dict['Z-Score'],'%')


# Min Max Scaler on the data
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(reshaping_df[
    ['Room_minmax','crime_C_avg_minmax','travel_min_to_CBD_minmax','Age_minmax']],reshaping_df['Price'],random_state=100)
# Developing LinearRegression model
lr_model_3 = LinearRegression()
# Fitting the model into our trained data
lr_model_3.fit(X_train_lr,y_train_lr)
# Calculating R-Squared value
accuracy_dict['MinMax'] = round(lr_model_3.score(X_test_lr,y_test_lr)*100,2)
print ('R-squared for the data after scaling with MinMax Scaler using LinearRegression model =',accuracy_dict['MinMax'],'%')


# Root transformation on the data
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(reshaping_df[['rooms_sqrt','crime_C_average_sqrt','travel_min_to_CBD_sqrt',
                                                                'age_sqrt']],reshaping_df['price_sqrt'],random_state=100)
# Developing LinearRegression model
lr_model_4 = LinearRegression()
# Fitting the model into our trained data
lr_model_4.fit(X_train_lr,y_train_lr)
# Calculating R-Squared value
accuracy_dict['Root transform'] = round(lr_model_4.score(X_test_lr,y_test_lr)*100,2)
print ('R-squared for the data after scaling with square root transform using LinearRegression model = ',accuracy_dict['Root transform'],'%')


# Squared transformation on the data
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(reshaping_df[['rooms_pwr','crime_C_average_pwr','age_pwr']],
                                                                reshaping_df['price_pwr'],random_state=100)
# Developing LinearRegression model
lr_model_5 = LinearRegression()
# Fitting the model into our trained data
lr_model_5.fit(X_train_lr,y_train_lr)
# Calculating R-Squared value
accuracy_dict['Squared transform'] = round(lr_model_5.score(X_test_lr,y_test_lr)*100,2)
print ('R-squared for the data after scaling with square powered using LinearRegression model = ',accuracy_dict['Squared transform'],'%')


# Log transformation on the data
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(reshaping_df[['rooms_log','crime_C_average_log','travel_min_to_CBD_log',
                                                                'age_log']],reshaping_df['price_log'],random_state=100)
# Developing LinearRegression model
lr_model_6 = LinearRegression()
# Fitting the model into our trained data
lr_model_6.fit(X_train_lr,y_train_lr)
# Calculating R-Squared value
accuracy_dict['Log transform'] = round(lr_model_6.score(X_test_lr,y_test_lr)*100,2)
print ('R-squared for the data after scaling with Log using LinearRegression model = ',accuracy_dict['Log transform'],'%')


After performing all kinds of normalisation & transformation techniques with their accuracies, we had plotted all accuracies of all the techniques performed above with its R-squared value using a `Bar chart` to give the better prediction linear model.

In [ ]:
# Plotting Bar chart to compare accuracies with all transformation techniques
plt.bar(range(len(accuracy_dict)), list(accuracy_dict.values()), align='center')
plt.xticks(range(len(accuracy_dict)), list(accuracy_dict.keys()),rotation=90)
plt.ylim(0,50)
plt.xlabel('Transformation')
plt.ylabel('Accuracy')
plt.title('Transformation Vs Accuracies');

From the above graph we can confirmly say that the **`Log Transformation`** performed way better than Square root transformation in terms of accuracy compared to other transformations as it reduces the variance in the data making it a better data for the linear model, which will predict the accuracy much better than other models.


**Conclusion:** So i conclude that the **Log transformation** is the prefered transformation for the given data. 

### Errors Found in the given data:

1. In the given data there are duplicates in ID of a given property, as the data becomes baised and inaccurate because of these duplicates. Hence these duplicates must be removed so that the ID will be unique for given property.

2. Incorrect column names, there are a few columns in the data with a wrong spelling. So therefore corrected those columns and named it as per the names given in the specifications file

3. For some rows the departure time from a station is greater than 24, According to reference `the departure time is A service day is a time period used to indicate route scheduling, A service day may exceed 24:00:00 if service begins on one day and ends on a following day. For example, service that runs from 08:00:00 on Friday to 02:00:00 on Saturday, could be denoted as running from 08:00:00 to 26:00:00 on a single service day.` So to avoid errors when converting the time column from string to date time, we drop those rows which have departure time greater than 24.

4. When we are crawling the data from the website mentioned for the information about secondary school, The website asks you for login credientials of monash every time you open it, as this becomes a security issue when crawling. To overcome this we copied the source code of the page into xml file and named it as website2.xml for parsing.

5. There are few duplicates in the school names, As the school names may be same for different areas but as the area is not mentioned in the data. There will be error in retriving a rank for a particular school, to avoid this we drop the duplicates from the data. 

## References

- The `pandas` Project. (2016a). *pandas 0.19.2 documentation: pandas.DataFrame.to_csv*. Retrieved from http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html#pandas.DataFrame.to_csv
- The `re` documentation - *re — Regular expression operations*. Retrieved from https://docs.python.org/3/library/re.html
* numpy (for calcualting linear algebraic equations, included in Python 3.7) https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html
* matplotlib https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.boxplot.html
* datetime https://docs.python.org/3/library/datetime.html
* Linear Regression for prediction using Linear models https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
* R2-Score (for calculating R-squared value) https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
* https://developers.google.com/transit/gtfs/reference
* https://stackoverflow.com/questions/52968877/read-xml-file-to-pandas-dataframe
* https://stackoverflow.com/questions/52690994/web-scraping-python-writing-to-a-csv
* https://gtfs.org/
* https://en.wikipedia.org/wiki/General_Transit_Feed_Specification
* https://dev.to/rokaandy/logarithmic-transformation-in-linear-regression-models-why-when-3a7c
* https://stackoverflow.com/questions/32108179/linear-regression-normalization-vs-standardization
* https://stats.stackexchange.com/questions/2142/linear-regression-effect-sizes-when-using-transformed-variables